In [0]:
import pandas as pd
import librosa
import numpy as np
from sklearn import preprocessing
from keras.utils import np_utils

train=pd.read_csv('drive/My Drive/Dokumente/Hackbay/hackbay_soundClassification/datasets/train.csv')

def parser(row):
    #print(row)
    print(row.ID)
    try:
        filename='drive/My Drive/Dokumente/Hackbay/hackbay_soundClassification/datasets/Train/Train/'+str(row.ID)+'.wav'
        x,s=librosa.load(filename,res_type='kaiser_fast')
        mfccs=np.mean(librosa.feature.mfcc(y=x,sr=s,n_mfcc=40).T,axis=0)
    except Exception as e:
        #print('Error in ',filename)
        #print(e)
        return None,None
    
    feature=mfccs
    label=row.Class
    return [feature,label]

temp=train.apply(parser,axis=1)
#temp.columns=['feature','label']

col = ["feature","label"]

newTemp = temp.tolist()
newDf = pd.DataFrame(newTemp, columns = col)

X=np.array(newDf.feature.tolist())
Y=np.array(newDf.label.tolist())

print("Done")

lb=preprocessing.LabelEncoder()
yy=np_utils.to_categorical(lb.fit_transform(Y))



In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from sklearn import metrics 
import warnings

warnings.filterwarnings("ignore")


num_labels = yy.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

print("Done")

Done


In [23]:
from sklearn.model_selection import train_test_split

aTrain,aTest,bTrain,bTest=train_test_split(X,yy,test_size=0.2)
model.fit(aTrain, bTrain, batch_size=32, validation_data=(aTest, bTest), epochs = 10)

Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 0s 224us/step - loss: 0.5242 - acc: 0.9500 - val_loss: 0.4030 - val_acc: 0.9750
Epoch 2/10
160/160 [==============================] - 0s 176us/step - loss: 0.4358 - acc: 0.9688 - val_loss: 0.4043 - val_acc: 0.9750
Epoch 3/10
160/160 [==============================] - 0s 163us/step - loss: 0.3695 - acc: 0.9750 - val_loss: 0.4209 - val_acc: 0.9500
Epoch 4/10
160/160 [==============================] - 0s 167us/step - loss: 0.1955 - acc: 0.9813 - val_loss: 0.4082 - val_acc: 0.9750
Epoch 5/10
160/160 [==============================] - 0s 186us/step - loss: 0.5974 - acc: 0.9562 - val_loss: 0.4250 - val_acc: 0.9500
Epoch 6/10
160/160 [==============================] - 0s 159us/step - loss: 0.3781 - acc: 0.9625 - val_loss: 0.7088 - val_acc: 0.9500
Epoch 7/10
160/160 [==============================] - 0s 169us/step - loss: 0.3806 - acc: 0.9688 - val_loss: 0.9964 - val_acc: 0.9250
Epoch 8/10
160/16

In [0]:
model.save('hailNN.h5')  # creates a HDF5 file 'hailNN.h5'